# Binary Classification of Survival Rates from Emergency Room Visits

#### XGBoost, no graphs

## Rebecca Stewart



### Overview

The challenge is to create a model that uses data from the first 24 hours of intensive care to predict patient survival. MIT's GOSSIS community initiative, with privacy certification from the Harvard Privacy Lab, has provided a dataset of more than 130,000 hospital Intensive Care Unit (ICU) visits from patients, spanning a one-year timeframe. This data is part of a growing global effort and consortium spanning Argentina, Australia, New Zealand, Sri Lanka, Brazil, and more than 200 hospitals in the United States.

Labeled training data are provided for model development; you will then upload your predictions for unlabeled data to Kaggle and these predictions will be used to determine the public leaderboard rankings, as well as the final winners of the competition.

Data analysis can be completed using your preferred tools. Tutorials, sample code, and other resources will be posted throughout the competition at widsconference.org/datathon and on the Kaggle Discussion Forum. The winners will be determined by the leaderboard on the Kaggle platform at the time the contest closes February 24.

### Import Libraries 

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import gc
import string
import time
import spacy
from collections import Counter
import scipy.stats as ss
from scipy import stats
import lightgbm as lgb 
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE 

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
sns.set()
%matplotlib inline

Using TensorFlow backend.
C:\Users\rebec\Anaconda3\envs\uwdatasci420\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
max_records=5000


#pip install xgboost
#setting display options
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', 500)
np.set_printoptions(linewidth =400)

### Helper Functions 
The first one, reduce_mem_useage, was taken from Kaggle.  Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage.

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def predict_evaluate(X_test, y_test, classifier):
    y_pred = classifier.predict(X_test)
    yhat = classifier.predict_proba(X_test)
    pos_probs = yhat[:, 1]
    print("Confusion Matrix")
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    acc_score=accuracy_score(y_test, y_pred)
    print("Accuracy Score " + str(acc_score))
    fpr, tpr, thresholds = roc_curve(y_test, pos_probs)
    
    roc_auc = roc_auc_score(y_test, pos_probs)
    plot_roc_curve(fpr, tpr, roc_auc)
    print("AUC " + str(roc_auc))
    
def plot_roc_curve(fpr, tpr, auc):
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label='ROC curve (area = %.3f)'%auc)
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()
    


<h3>Load the Dataset</h3>

In [4]:
# Load Dataset  -- location is as follows
#https://www.kaggle.com/c/widsdatathon2020/download/RJVMMbUOhozhdYOtAUtz%2Fversions%2FwogAJBiaORb3PdRnSB0M%2Ffiles%2Fsamplesubmission.csv
# downloaded on 1/13/2020 at 9:20 am

load_unlabled_data = False
limit_dataset_size = True

## Read the .csv file with the pandas read_csv method an create a dataframe
tr_data = pd.read_csv("training_v2.csv")

if load_unlabled_data: ul_data = pd.read_csv("unlabeled.csv")

info_df = pd.read_csv('WiDS Datathon 2020 Dictionary.csv')
tr_data.head(5)

if limit_dataset_size: 
    tr_data = tr_data.head(max_records)
    # tr_data = resample(tr_data, replace=False, n_samples=max_records) 
    
tr_data.shape

(91713, 186)

In [5]:
gc.collect()
tr_data.drop(['encounter_id', 'patient_id'], axis=1, inplace=True)
tr_data.drop(['readmission_status'], axis=1, inplace=True)

### Integer / Binary Columns

Before converting to int or bool, we need to figure out what to do with missing values.

#### Drop features with a large number of missing values

Let's take a look at those columns that have more than 80% missing values

In [6]:
# Create a dataframe of the columns that have more than a certain percentage missing values
pct_cutt_off =  .8
isna_counts = tr_data.isna().sum()
isna_counts = pd.DataFrame({'col':isna_counts.index, 'cnt':isna_counts.values})
isna_counts["pct"] = isna_counts['cnt']/tr_data.shape[0]
isna_counts = isna_counts[isna_counts.loc[:, "pct"] > pct_cutt_off] 
isna_counts.sort_values(by=['col']).head(100)
drop_cols = list(isna_counts['col'])


In [7]:
# DROP THESE COLUMNS
tr_data.drop(drop_cols, axis=1, inplace=True)
if load_unlabled_data: ul_data.drop(drop_cols, axis=1, inplace=True)
  

#### Impute Integer Features

For those features that should be of type integer or boolean, impute with most frequent before converting to type integer.

In [8]:
# Before imputing all missing values, let's deal with these by themselves
# Before converting to integer, we will replace missing values with the mode
int_cols=['gcs_eyes_apache', 'gcs_motor_apache', 'gcs_verbal_apache', 'heart_rate_apache', 'map_apache', 'resprate_apache']
int_cols=int_cols + ['d1_diasbp_max', 'd1_diasbp_min', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min', 'd1_resprate_max', 'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max', 'd1_sysbp_min', 'h1_diasbp_max']
int_cols=int_cols + ['h1_diasbp_min', 'h1_heartrate_max', 'h1_heartrate_min', 'h1_mbp_max', 'h1_mbp_min', 'h1_resprate_max', 'h1_resprate_min', 'h1_spo2_max', 'h1_spo2_min', 'h1_sysbp_max', 'h1_sysbp_min', 'd1_glucose_max', 'd1_glucose_min']
int_cols=int_cols +['arf_apache', 'gcs_unable_apache', 'intubated_apache', 'ventilated_apache', 'aids', 'cirrhosis', 'diabetes_mellitus']
int_cols=int_cols + ['hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']

imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
tr_data[int_cols]  = imputer.fit_transform(tr_data[int_cols])
tr_data[int_cols] = tr_data[int_cols].astype(int)

if load_unlabled_data: ul_data[int_cols]  = imputer.fit_transform(ul_data[int_cols])
if load_unlabled_data: ul_data[int_cols] = ul_data[int_cols].astype(int)

### String/Object Columns

Next we will take a look at string/categorical features. Which ones contain missing values or values that can be merged?

In [9]:
str_cols=['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source',  'icu_stay_type', 'icu_type', 'apache_2_diagnosis', 'apache_3j_diagnosis', 'apache_3j_bodysystem', 'apache_2_bodysystem', 'apache_2_diagnosis', 'apache_3j_diagnosis']
tr_data[str_cols] = tr_data[str_cols].astype('str')
if load_unlabled_data: ul_data[str_cols] = ul_data[str_cols].astype('str')


### Clean Up the Data

Next we will take a look at string/categorical features. Which ones contain missing values or values that can be merged?

In [10]:
# Let's deal with missing values in our categorical features, like gender, ethnicity, etc.
tr_data["gender"].fillna("NA", inplace = True)  
tr_data["ethnicity"].fillna("Other/Unknown", inplace = True)  
tr_data["apache_2_bodysystem"].fillna("Undefined Diagnoses", inplace = True)  
tr_data["apache_3j_bodysystem"].fillna("Other", inplace = True) 
tr_data["icu_admit_source"].fillna("Other", inplace = True) 
tr_data["hospital_admit_source"].fillna("Other", inplace = True) 

# These should really be the same value
tr_data.loc[tr_data.loc[:, "apache_2_bodysystem"] == "Undefined diagnoses", "apache_2_bodysystem"] = "Undefined Diagnoses"
tr_data.loc[tr_data.loc[:, "apache_2_diagnosis"] == "185.40173901455842", "apache_2_diagnosis"] = "185.0"


if load_unlabled_data: ul_data["gender"].fillna("NA", inplace = True)  
if load_unlabled_data: ul_data["ethnicity"].fillna("Other/Unknown", inplace = True)  
if load_unlabled_data: ul_data["apache_2_bodysystem"].fillna("Undefined Diagnoses", inplace = True)  
if load_unlabled_data: ul_data["apache_3j_bodysystem"].fillna("Other", inplace = True) 
if load_unlabled_data: ul_data["icu_admit_source"].fillna("Other", inplace = True) 
if load_unlabled_data: ul_data["hospital_admit_source"].fillna("Other", inplace = True) 

# These should really be the same value
if load_unlabled_data: ul_data.loc[ul_data.loc[:, "apache_2_bodysystem"] == "Undefined diagnoses", "apache_2_bodysystem"] = "Undefined Diagnoses"
if load_unlabled_data: ul_data.loc[ul_data.loc[:, "apache_2_diagnosis"] == "185.40173901455842", "apache_2_diagnosis"] = "185.0"



### Feature Engineering

Length of stay in ICU should not be negative, but, before we clean up the data, let's create a new column 

In [11]:

tr_data.loc[tr_data.loc[:, "pre_icu_los_days"] <0, "readmitted_to_icu"] = 1
tr_data.loc[tr_data.loc[:, "pre_icu_los_days"] >=0, "readmitted_to_icu"] = 0
tr_data.loc[tr_data.loc[:, "pre_icu_los_days"] <0, "pre_icu_los_days"] = 0

if load_unlabled_data: 
    ul_data.loc[ul_data.loc[:, "pre_icu_los_days"] <0, "pre_icu_los_days"] = 0
    ul_data.loc[ul_data.loc[:, "pre_icu_los_days"] <0, "readmitted_to_icu"] = 1
    ul_data.loc[ul_data.loc[:, "pre_icu_los_days"] >=0, "readmitted_to_icu"] = 0    

### Impute Missing Values for Float Columns

In [12]:
for col in tr_data.columns:
    if tr_data[col].dtypes == 'float64':
        if tr_data[col].isna().sum()>0:
            HasNan = np.isnan(tr_data[col])
            tr_data.loc[HasNan, col] = np.mean(tr_data[col])
        if load_unlabled_data:
            if ul_data[col].isna().sum()>0:
                HasNan = np.isnan(ul_data[col])
                ul_data.loc[HasNan, col] = np.mean(ul_data[col])

### Replace Outliers for Float Datatypes

In [13]:
replace_outliers=True
if replace_outliers:
    for col in tr_data.columns:
        if tr_data[col].dtypes in ['float64', 'float16']:
            TooHighOrLow =  (tr_data.loc[:, col] >   tr_data[col].mean() + 3 * tr_data[col].std()) | (tr_data.loc[:, col] <   tr_data[col].mean() - 3 * tr_data[col].std())
            tr_data.loc[TooHighOrLow, col] = np.median(tr_data.loc[~TooHighOrLow,col])
        


In [14]:
tr_data = reduce_mem_usage(tr_data)
if load_unlabled_data: ul_data = reduce_mem_usage(ul_data)
tr_data.dtypes.value_counts().sort_index()

Mem. usage decreased to 28.95 Mb (67.7% reduction)


int8       29
float16    91
int16      20
object     10
dtype: int64

In [16]:
tr_data.shape

(91713, 150)

### Separate Target Variable

In [ ]:
target_label='hospital_death'
X = tr_data.drop([target_label], axis=1)
y = tr_data[target_label]

if load_unlabled_data: X_ul = ul_data.drop([target_label], axis=1)

<h3>One Hot Encode the Data</h3>

In [ ]:
train_objs_num = len(X)
if load_unlabled_data: 
    dataset = pd.concat(objs=[X, X_ul], axis=0)
else:
    dataset=X
    
dataset_preprocessed = pd.get_dummies(dataset)
columns = dataset_preprocessed.columns

if load_unlabled_data:
    X = dataset_preprocessed[:train_objs_num]
    X_ul = dataset_preprocessed[train_objs_num:]
else:
    X = dataset_preprocessed

<h3>Scale the Data</h3>

In [ ]:
X = preprocessing.scale(X)
if load_unlabled_data: X_ul = preprocessing.scale(X_ul)

### Split the Data Test/Train

We will split our newly vectorized data into 80% train and 20% test. Our random state value will ensure that the data is split the same way each time so that we can replicate results. 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=22)

In [ ]:
param_dist = {'objective':'binary:logistic', 'n_estimators':100, 'learning_rate':0.01, 'colsample_bytree': 0.8,
 'gamma': 0.5,  'max_depth': 3, 'min_child_weight': 3, 'subsample': 0.8}

clf = xgb.XGBClassifier(**param_dist)

clf.fit(x_train, y_train,
        eval_set=[(x_train, y_train), (x_test, y_test)],
        eval_metric='logloss',
        verbose=False)

evals_result = clf.evals_result()

In [ ]:
predict_evaluate(x_train, y_train, clf) 

In [ ]:
predict_evaluate(x_test, y_test, clf) 

<h3>Train the Model </h3>


In [ ]:
gc.collect()
'''


data_dmatrix = xgb.DMatrix(data=X,label=y)

estimator= xgb.XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',  silent=True, nthread=1)

parameters = {
    'max_depth': [3],
    'min_child_weight': [1,3,5],
    'learning_rate': [0.1, 0.01, 0.005],
    'subsample': [.8, 1.0],
    'colsample_bytree': [ .8,1.0],
    'gamma': [0.5]
}


grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

grid_search.fit(x_train, y_train)


end = time.time()
print(end - start)
'''

In [ ]:
# grid_search.best_estimator_

In [ ]:
# grid_search.best_params_

In [ ]:
# predict_evaluate(x_test, y_test, grid_search) 